# Conception BDD Normalisée et Conforme au RGPD (SIDORA AI)
---
## 

## CRUD pour verifier les donneés

In [13]:
# import
# import
from datetime import datetime
import pandas as pd
import numpy as np
from passlib.hash import argon2

from sqlalchemy import Column, Integer, String, Boolean, DateTime, ForeignKey, create_engine
from sqlalchemy.orm import relationship, declarative_base, sessionmaker
from sqlalchemy.sql import func

from faker import Faker

In [9]:
# db_reader.py
# создаем engine (укажите ваш путь к БД)
engine = create_engine("sqlite:///BD_Ventes_de_jeux_video.bd")  # или ваша база

# создаем сессию
Session = sessionmaker(bind=engine)
session = Session()

In [21]:
# Функция для чтения таблицы и возврата DataFrame
def read_table(table_class, limit=None, filter_exp=None):
    """
    table_class: class SQLAlchemy (par exemple.: Client, Commande)
    limit: opt
    """
    query = session.query(table_class)
    
    if filter_exp:
        query = query.filter(filter_exp)
    
    if limit:
        query = query.limit(limit)

    if query and table_class == "Client":
        # Обновляем дату последнего использования
        query.date_derniere_utilisation = datetime.utcnow()
        session.commit()  # сохраняем изменения
    df = pd.read_sql(query.statement, session.bind)
    return df

In [23]:
read_table(Client, filter_exp=Client.client_id==0)

,client_id,age_id,region_id,date_creation,date_derniere_utilisation
0,1,2,1,2025-11-30 10:01:57,2025-11-30 10:01:57
1,2,1,0,2025-11-30 10:01:57,2025-11-30 10:01:57
2,3,2,0,2025-11-30 10:01:57,2025-11-30 10:01:57
3,4,1,0,2025-11-30 10:01:57,2025-11-30 10:01:57
4,5,2,0,2025-11-30 10:01:58,2025-11-30 10:01:58
...,...,...,...,...,...
345,346,3,0,2025-11-30 10:02:26,2025-11-30 10:02:26
346,347,0,2,2025-11-30 10:02:26,2025-11-30 10:02:26
347,348,2,2,2025-11-30 10:02:27,2025-11-30 10:02:27
348,349,2,1,2025-11-30 10:02:27,2025-11-30 10:02:27


In [12]:
session.close()
engine.dispose()